In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.constraints import *
from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.table.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

In [10]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

Meshcat url:  http://127.0.0.1:7007/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [46]:
# Set up grammar and constraint set.
grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint(),
    TallStackConstraint(),
    NumStacksConstraint()
]

def sample_realistic_scene(grammar, constraints, seed=None, skip_physics_constraints=False):
    if seed is not None:
        torch.random.manual_seed(seed)
    structure_constraints, pose_constraints = split_constraints(constraints)
    if len(structure_constraints) > 0:
        tree, success = rejection_sample_under_constraints(grammar, structure_constraints, 1000, detach=True)
        if not success:
            logging.error("Couldn't rejection sample a feasible tree config.")
            return None, None
    else:
        tree = grammar.sample_tree(detach=True)

    samples = do_fixed_structure_hmc_with_constraint_penalties(
        grammar, tree, num_samples=250, subsample_step=1,
        with_nonpenetration=False, zmq_url=vis.window.zmq_url,
        constraints=pose_constraints,
        kernel_type="NUTS", max_tree_depth=6, target_accept_prob=0.8, adapt_step_size=True,
        #kernel_type="HMC", num_steps=1, step_size=1E-1, adapt_step_size=False, # Langevin-ish
        structure_vis_kwargs={"with_triad": False, "linewidth": 30, "node_sphere_size": 0.02,
                             "alpha": 0.5}
    )

    # Step through samples backwards in HMC process and pick out a tree that satisfies
    # the constraints.
    good_tree = None
    best_bad_tree = None
    best_violation = None
    for candidate_tree in samples[::-1]:
        total_violation = eval_total_constraint_set_violation(candidate_tree, constraints)
        if total_violation <= 0.:
            good_tree = candidate_tree
            break
        else:
            if best_bad_tree is None or total_violation <= best_violation:
                best_bad_tree = candidate_tree
                best_violation = total_violation.detach()
            
    if good_tree == None:
        logging.error("No tree in samples satisfied constraints.")
        print("Best total violation: %f" % best_violation)
        print("Violations of best bad tree:")
        for constraint in constraints:
            print("constraint ", constraint, ": ", constraint.eval(best_bad_tree))
        return None, None

    if skip_physics_constraints:
        return None, good_tree

    feasible_tree = project_tree_to_feasibility(deepcopy(good_tree), do_forward_sim=True, timestep=0.001, T=1.)
    return feasible_tree, good_tree

In [47]:
test_tree, _ = sample_realistic_scene(grammar, constraints, seed=42)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")
# Save that scene out for use in another notebook
with open("example_feasible_sampled_scene.pickle", "wb") as f:
    pickle.dump(test_tree, f)

Initial trace log prob:  tensor(-18710.0871)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample:  20%|██        | 61/300 [05:16,  5.18s/it, step size=3.38e-02, acc. prob=0.784]
ERROR:root:No tree in samples satisfied constraints.


Best total violation: 0.006404
Violations of best bad tree:
constraint  <spatial_scene_grammars_examples.table.grammar.ObjectsOnTableConstraint object at 0x7f7929484ba8> :  tensor([[-0.1099,  0.1249,  0.0081],
        [-0.2326, -0.2326,  0.0264],
        [ 0.1279,  0.2207,  0.0262],
        [ 0.0494, -0.0242,  0.0345],
        [ 0.2443, -0.1689,  0.0153],
        [-0.4714,  0.0244,  0.0292],
        [-0.2616,  0.0499,  0.0333],
        [ 0.4729,  0.0650,  0.0413],
        [ 0.2661,  0.0415,  0.0049],
        [ 0.0191,  0.4702,  0.0260],
        [-0.0839,  0.2872,  0.0072],
        [ 0.0078, -0.4746,  0.0188],
        [ 0.0469, -0.2748,  0.0106],
        [ 0.1378,  0.2271,  0.1261],
        [ 0.2478, -0.1545,  0.1357],
        [ 0.2554, -0.1281,  0.2306],
        [ 0.2548, -0.1223,  0.3157]])
constraint  <spatial_scene_grammars_examples.table.grammar.ObjectSpacingConstraint object at 0x7f792964fc50> :  tensor([[ 5.2881e-02],
        [ 1.7317e-02],
        [ 2.3307e-01],
        [-8.1143

In [5]:
# Produce dataset by sampling a bunch of environments.
# Save grammar state dict
#state_dict_file = "target_dataset_grammar_state_dict.torch"
state_dict_file = "structure_constraint_dataset_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
#dataset_save_file = "target_dataset_examples.pickle"
dataset_save_file = "structure_constraint_examples.pickle"
N = 20
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    tree, _ = sample_realistic_scene(grammar, constraints)
    if tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump(tree, f)
        k += 1
        pbar.update(k)
pbar.close()

Saving state dict to  structure_constraint_dataset_grammar_state_dict.torch


Samples:   0%|          | 0/20 [00:00<?, ?it/s]

Initial trace log prob:  tensor(-3213.9969)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:37,  2.63s/it, step size=8.38e-04, acc. prob=0.926]


Initial trace log prob:  tensor(-21753.3193)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


...

Sample: 100%|██████████| 37/37 [01:52,  3.04s/it, step size=1.25e-03, acc. prob=0.758]


Logfile: 

         S N O P T  7.4-1.2  (Feb 2015)
1
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
1
 
 Parameters

 Files
 -----
 Solution file..........         0       Old basis file ........         0       Standard input.........         5
 Insert file............         0       New basis file ........         0       (Printer)..............        10
 Punch file.............         0       Backup basis file......         0       (Specs file)...........         0
 Load file..............         0       Dump file..............         0       Standard output........         6

 Frequencies
 -----------
 Print frequency........       100       Check frequency........        60       Save new basis map.....       100
 Summary frequency......       100       Factorization frequency        50       Expand frequency.......     10000

 QP subproblems
 --------------
 QPsolver Cholesky......
 Scale tolerance........     0.900       Minor 

Initial trace log prob:  tensor(-14600.1775)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:52,  3.05s/it, step size=7.09e-04, acc. prob=0.941]


KeyboardInterrupt: 

In [ ]:
print(tree.score())

# Build sample population from various stages of em fitting process

In [6]:
fit_grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint()
]
#fit_grammar.load_state_dict(torch.load("fit_grammar.torch"))
with open("fit_em.pickle", "rb") as fp:
    em = pickle.load(fp)
fit_grammar.load_state_dict(em.grammar_iters[-1])

<All keys matched successfully>

In [7]:
test_tree, _ = sample_realistic_scene(fit_grammar, constraints, seed=43)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")

Initial trace log prob:  tensor(-298.1362)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:09,  1.89s/it, step size=7.15e-04, acc. prob=0.910]


In [8]:
# This one saves out the pre-fit examples, and doesn't bother with physics,
# since these samples are all over the place. It still does HMC to try to resolve
# the basic constraints (for better comparison with post-fit)
fit_grammar.load_state_dict(em.grammar_iters[0])
SKIP_PHYSICS_CONSTRAINTS = True
state_dict_file = "pre_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "pre_fit_grammar_draws.pickle"
N = 50
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    print(k, N)
    _, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints)
    if nonfeasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((None, nonfeasible_tree), f)
        k += 1
        pbar.update(k)
pbar.close()

Saving state dict to  pre_fit_grammar_state_dict.torch


Samples:   0%|          | 0/50 [00:00<?, ?it/s]

0 50
Initial trace log prob:  tensor(-38490375.2478)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:06,  1.79s/it, step size=8.66e-04, acc. prob=0.941]


1 50
Initial trace log prob:  tensor(-35593269.8138)


...

Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:38,  1.05s/it, step size=9.16e-04, acc. prob=0.948]


49 50
Initial trace log prob:  tensor(-12347141.8185)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:49,  1.34s/it, step size=7.52e-04, acc. prob=0.972]


In [9]:
# This one saves out the post-fit examples, before and after the physics projection step.
SKIP_PHYSICS_CONSTRAINTS = True
fit_grammar.load_state_dict(em.grammar_iters[-1])
state_dict_file = "post_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "post_fit_grammar_draws.pickle"
N = 50
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    feasible_tree, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints)
    if feasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((feasible_tree, nonfeasible_tree), f)
        k = k + 1
        pbar.update(k)
pbar.close()

Saving state dict to  post_fit_grammar_state_dict.torch


Samples:   0%|          | 0/50 [00:00<?, ?it/s]

Initial trace log prob:  tensor(-11952.1830)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:59,  3.23s/it, step size=6.02e-04, acc. prob=0.908]


Logfile: 

         S N O P T  7.4-1.2  (Feb 2015)
1
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
1
 
 Parameters

 Files
 -----
 Solution file..........         0       Old basis file ........         0       Standard input.........         5
 Insert file............         0       New basis file ........         0       (Printer)..............        10
 Punch file.............         0       Backup basis file......         0       (Specs file)...........         0
 Load file..............         0       Dump file..............         0       Standard output........         6

 Frequencies
 -----------
 Print frequency........       100       Check frequency........        60       Save new basis map.....       100
 Summary frequency......       100       Factorization frequency        50       Expand frequency.......     10000

 QP subproblems
 --------------
 QPsolver Cholesky......
 Scale tolerance........     0.900       Minor 

...

Sample: 100%|██████████| 37/37 [01:13,  1.97s/it, step size=7.79e-04, acc. prob=0.832]


Initial trace log prob:  tensor(-1269.4815)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:54,  1.47s/it, step size=9.29e-04, acc. prob=0.849]


# Same thing, but baseline grammar

In [14]:
from spatial_scene_grammars_examples.table.grammar_baseline import *

fit_grammar = SpatialSceneGrammar(
    root_node_type = TableBaseline,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint()
]
#fit_grammar.load_state_dict(torch.load("fit_grammar.torch"))
with open("fit_em_baseline.pickle", "rb") as fp:
    em = pickle.load(fp)
fit_grammar.load_state_dict(em.grammar_iters[-1])

<All keys matched successfully>

In [12]:
test_tree, _ = sample_realistic_scene(fit_grammar, constraints, seed=43)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")

Initial trace log prob:  tensor(112.9753)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:54,  1.48s/it, step size=8.57e-04, acc. prob=0.862]


In [13]:
# This one saves out the pre-fit examples, and doesn't bother with physics,
# since these samples are all over the place. It still does HMC to try to resolve
# the basic constraints (for better comparison with post-fit)
fit_grammar.load_state_dict(em.grammar_iters[0])
SKIP_PHYSICS_CONSTRAINTS = True
state_dict_file = "baseline_pre_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "baseline_pre_fit_grammar_draws.pickle"
N = 30
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    print(k, N)
    _, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints)
    if nonfeasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((None, nonfeasible_tree), f)
        k += 1
        pbar.update(k)
pbar.close()

Saving state dict to  baseline_pre_fit_grammar_state_dict.torch


Samples:   0%|          | 0/30 [00:00<?, ?it/s]

0 30
Initial trace log prob:  tensor(-30.7523)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


...

16 30
Initial trace log prob:  tensor(-52.2274)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Warmup:  19%|█▉        | 7/37 [00:05,  1.23it/s, step size=1.98e-01, acc. prob=0.684]

KeyboardInterrupt: 

In [16]:
# This one saves out the post-fit examples, before and after the physics projection step.
SKIP_PHYSICS_CONSTRAINTS = True
fit_grammar.load_state_dict(em.grammar_iters[-1])
state_dict_file = "baseline_post_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "baseline_post_fit_grammar_draws.pickle"
N = 100
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    feasible_tree, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints)
    if feasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((feasible_tree, nonfeasible_tree), f)
        k = k + 1
        pbar.update(k)
pbar.close()

Saving state dict to  baseline_post_fit_grammar_state_dict.torch


Samples:   0%|          | 0/100 [00:00<?, ?it/s]

Initial trace log prob:  tensor(109.0822)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


...

Initial trace log prob:  tensor(91.5527)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:26,  2.34s/it, step size=7.35e-04, acc. prob=0.907]


Initial trace log prob:  tensor(78.6903)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.


Sample:  92%|█████████▏| 34/37 [01:06,  2.52s/it, step size=7.90e-04, acc. prob=0.885]

KeyboardInterrupt: 